### [1292 元素和小于等于阈值的正方形的最大边长](https://leetcode.cn/problems/maximum-side-length-of-a-square-with-sum-less-than-or-equal-to-threshold/description/?envType=daily-question&envId=2026-01-19)  
<font color=#FFCC33>中等</font>  
本题 和 [Solution1895](./Solution1895.ipynb) 有些类似

但如果使用一维前缀和会 TLE, 复杂度过高

In [ ]:
from typing import List


class Solution:
    def maxSideLength(self, mat: List[List[int]], threshold: int) -> int:
        row, column = len(mat), len(mat[0])

        row_sum = [[0] * (column + 1) for _ in range(row + 1)]
        for i in range(1, row + 1):
            for j in range(1, column + 1):
                row_sum[i][j] = row_sum[i][j - 1] + mat[i - 1][j - 1]

        side = min(row, column)
        for l in range(side, 0, -1):
            for i in range(row-l+1):
                for j in range(column-l+1):
                    tmp = 0
                    for c in range(l):
                        tmp += row_sum[i + c + 1][j + l] - row_sum[i + c + 1][j]
                    if tmp <= threshold:
                        return l
        return 0

一种优化思路是使用二维前缀和:
很好，这题正好是**二维前缀和**最典型、也最值得掌握的入门场景。我按「**是什么 → 怎么构建 → 怎么用 → 怎么记**」四个层次给你讲清楚。

---

## 一、二维前缀和是什么（一句话版）

**二维前缀和**就是：

> 用一个额外的矩阵 `pre`，让你在 **O(1)** 时间内求出任意一个**子矩形的元素和**。

它是你已经熟悉的「一维前缀和」在二维矩阵上的**直接推广**。

---

## 二、从你熟悉的一维前缀和开始类比

### 1️⃣ 一维前缀和你一定见过

原数组：

```
a = [2, 3, 1, 4]
```

前缀和：

```
pre[i] = a[0] + a[1] + ... + a[i-1]
pre = [0, 2, 5, 6, 10]
```

区间和 `[l, r]`：

```
sum(l, r) = pre[r+1] - pre[l]
```

> 关键记忆点：**区间 = 右前缀 − 左前缀**

---

## 三、二维前缀和的定义（核心）

### 2️⃣ 定义一个二维 `pre`

给定矩阵 `mat`，我们定义：

```
pre[i][j] =
mat 中左上角 (0,0) 到 (i-1, j-1) 这个矩形的和
```

👉 **注意是 (i-1, j-1)，所以 pre 比 mat 多一行一列**

```
pre 大小是 (m+1) x (n+1)
```

---

## 四、二维前缀和怎么构建

### 3️⃣ 构建公式（非常重要）

```
pre[i][j] =
    pre[i-1][j]      # 上方矩形
  + pre[i][j-1]      # 左方矩形
  - pre[i-1][j-1]    # 重复计算的左上角
  + mat[i-1][j-1]    # 当前点
```

### 直观图解

```
+-------+
|   A   |
+---+---+
|   | x |
+---+---+
```

* `A` 被算了两次，要减一次
* `x` 是新加入的点

---

### 代码模板（必须背）

```python
m, n = len(mat), len(mat[0])
pre = [[0] * (n + 1) for _ in range(m + 1)]

for i in range(1, m + 1):
    for j in range(1, n + 1):
        pre[i][j] = (
            pre[i-1][j]
            + pre[i][j-1]
            - pre[i-1][j-1]
            + mat[i-1][j-1]
        )
```

---

## 五、二维前缀和怎么用（最关键）

### 4️⃣ 任意子矩形求和（核心公式）

要求矩形：

```
左上角: (r1, c1)
右下角: (r2, c2)
```

（全部是 **0-based**，且包含边界）

公式是：

```
sum =
  pre[r2+1][c2+1]
- pre[r1][c2+1]
- pre[r2+1][c1]
+ pre[r1][c1]
```

---

### 为什么是这个公式？（你要能画出来）

```
+-----------+
|     A     |
+-----+-----+
|  B  |  C  |
+-----+-----+
```

* 大矩形 A+B+C
* 减掉上方 A
* 减掉左方 B
* 左上角 A 被减多了，加回来

👉 **二维就是“一维前缀和 + 容斥原理”**

---

## 六、把它直接套到你这道题上

### 5️⃣ k × k 正方形的和

左上角是 `(i, j)`，边长是 `k`：

```
右下角是 (i+k-1, j+k-1)
```

代入公式：

```python
def square_sum(i, j, k):
    return (
        pre[i+k][j+k]
        - pre[i][j+k]
        - pre[i+k][j]
        + pre[i][j]
    )
```

⚠️ 注意：`pre` 的坐标是 **自动 +1 的版本**

---

## 七、为什么二维前缀和是这题的“必备技能”

你原来的写法：

* 每个正方形要 **按行累加 k 次**
* 时间复杂度 `O(k)`

二维前缀和：

* **每个正方形 O(1)**
* 才能支撑 `m*n*log(min(m,n))` 这种量级

这是这题从 **“暴力思维” → “算法思维”** 的分水岭。

---

## 八、记忆方法（给你一个「肌肉记忆口诀」）

### 口诀一（构建）

> **上 + 左 − 左上 + 自己**

### 口诀二（查询）

> **右下 − 上 − 左 + 左上**

### 口诀三（边界）

> **pre 永远比原矩阵大一圈**

---

## 九、什么时候你“应该想到二维前缀和”

只要题目出现这些关键词之一，你就该条件反射：

* 子矩阵和 / 子正方形和
* 多次区域求和
* 固定形状在矩阵里滑动
* `sum <= threshold` / `sum == k`

这道题是 **二维前缀和的教科书级入口题**，你之前没遇到是完全正常的，但现在这一步跨过去，后面很多矩阵题都会突然变简单。

In [ ]:
class Solution:
    def maxSideLength(self, mat: List[List[int]], threshold: int) -> int:
        m, n = len(mat), len(mat[0])
        s = [[0] * (n + 1) for _ in range(m + 1)]
        for i, row in enumerate(mat):
            for j, x in enumerate(row):
                s[i + 1][j + 1] = s[i + 1][j] + s[i][j + 1] - s[i][j] + x

        # 返回左上角在 (r1, c1)，右下角在 (r2, c2) 的子矩阵元素和
        def query(r1: int, c1: int, r2: int, c2: int) -> int:
            return s[r2 + 1][c2 + 1] - s[r2 + 1][c1] - s[r1][c2 + 1] + s[r1][c1]

        ans = 0
        for i in range(m):
            for j in range(n):
                # 边长为 ans+1 的正方形，左上角在 (i, j)，右下角在 (i+ans, j+ans)
                while i + ans < m and j + ans < n and query(i, j, i + ans, j + ans) <= threshold:
                    ans += 1
        return ans